# Real Time Application

In [11]:
import tensorflow as tf
from tensorflow import keras
import cv2
import csv
import numpy as np

In [2]:
model = keras.models.load_model('model.h5')

In [3]:
img_test = cv2.imread("mrlEyes_2018_01/train/1/s0001_01843_0_0_1_0_0_01.png")

In [22]:
img_test = cv2.resize(img_test, (80,80), interpolation = cv2.INTER_AREA)

In [23]:
img_test = img_test/255.0

In [24]:
new_img = tf.data.Dataset.from_tensor_slices([img_test])

In [25]:
prediction = model.predict(new_img.batch(32))
prediction.flatten()[0]

0.56163526

In [4]:
xml_path = "haarcascade_xml/"

In [8]:
face_cascade = cv2.CascadeClassifier(xml_path + "haarcascade_frontalface_default.xml")
right_eye_cascade = cv2.CascadeClassifier(xml_path + "haarcascade_righteye_2splits.xml")
left_eye_cascade = cv2.CascadeClassifier(xml_path + "haarcascade_lefteye_2splits.xml")

In [9]:
dim = (80,80)

In [24]:
import time

In [38]:
cap = cv2.VideoCapture(0)

closed_count = 0
frame_count = 0
start = time.time()
while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()
    frame_count += 1
    
    # Our operations on the frame come here
    gray_single = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    gray = np.zeros_like(frame)
    gray[:,:,0] = gray_single
    gray[:,:,1] = gray_single
    gray[:,:,2] = gray_single
    
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    if len(faces) > 0:
        (x,y,w,h) = faces[0]

        #img = cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
        half_int = int(np.ceil(w/2))

        roi_gray_left = gray[y:y+h, x:x+half_int]
        roi_gray_right = gray[y:y+h, x+half_int:x+w]

        frame = cv2.rectangle(frame,(x,y),(x+half_int,y+h),(255,0,0),2) # left part
        frame = cv2.rectangle(frame,(x+half_int,y),(x+w,y+h),(0,255,0),2) # right part

        roi_color_left = frame[y:y+h, x:x+half_int]
        roi_color_right = frame[y:y+h, x+half_int:x+w]

        right_eyes = right_eye_cascade.detectMultiScale(roi_gray_left)
        left_eyes = left_eye_cascade.detectMultiScale(roi_gray_right)

        # check we have detected something on both sides
        if len(right_eyes) > 0 and len(left_eyes) > 0:
            (rx,ry,rw,rh) = right_eyes[0]
            (lx,ly,lw,lh) = left_eyes[0]


            cv2.rectangle(roi_color_left,(rx,ry),(rx+rw,ry+rh),(0,255,0),2)
            cv2.rectangle(roi_color_right,(lx,ly),(lx+lw,ly+lh),(0,255,0),2)

            #take right eye image
            right_eye = roi_gray_left[ry:ry+rh, rx:rx+rw]
            left_eye = roi_gray_right[ly:ly+lh, lx:lx+lw]
            
            #cv2.imshow('frame',left_eye)
            
            right_eye_resized = cv2.resize(right_eye, dim, interpolation = cv2.INTER_AREA)
            left_eye_resized = cv2.resize(left_eye, dim, interpolation = cv2.INTER_AREA)
            
            right_eye_resized = right_eye_resized/255.0
            left_eye_resized = left_eye_resized/255.0
            
            left_mirror = cv2.flip(left_eye_resized, 1)
            #cv2.imshow('frame',left_mirror)
            
            right_final = tf.data.Dataset.from_tensor_slices([right_eye_resized])
            left_final = tf.data.Dataset.from_tensor_slices([left_mirror])
            
            
            right_open = model.predict(right_final.batch(32))
            left_open = model.predict(left_final.batch(32))
            
            print("Right:", right_open, " Left:", left_open)
            
            if right_open[0][0] < 0.5 and left_open[0][0] < 0.5:
                closed_count += 1
                cv2.putText(frame,'CLOSED', 
                    (500,460), 
                    cv2.FONT_HERSHEY_SIMPLEX, 
                    1,
                    (0,0,255),
                    2)

                #print("Closed eyes found")
            else:
                closed_count = 0
                #print("Open eyes found")
            
            if closed_count >= 6:
                print("Pay attention!") #chicchirichi
            
    # Display the resulting frame
    cv2.imshow('frame',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        stop = time.time()
        break
        
cap.release()
cv2.waitKey(0)
cv2.destroyAllWindows()

Right: [[1.]]  Left: [[1.]]
Right: [[1.]]  Left: [[1.]]
Right: [[1.]]  Left: [[1.]]
Right: [[0.08466896]]  Left: [[0.191288]]
Right: [[0.31118047]]  Left: [[0.32016668]]
Right: [[0.21771386]]  Left: [[0.12608802]]
Right: [[0.0335758]]  Left: [[0.0906235]]
Right: [[0.00704652]]  Left: [[1.]]
Right: [[0.15202007]]  Left: [[0.99999225]]
Right: [[0.23679826]]  Left: [[0.39145204]]
Right: [[0.16074559]]  Left: [[1.]]
Right: [[0.9242117]]  Left: [[0.00105694]]
Right: [[0.31525284]]  Left: [[0.7551342]]
Right: [[1.]]  Left: [[1.]]
Right: [[1.]]  Left: [[1.]]
Right: [[1.]]  Left: [[1.]]
Right: [[1.]]  Left: [[1.]]
Right: [[0.05654818]]  Left: [[0.39618546]]
Right: [[0.00819734]]  Left: [[0.85923123]]
Right: [[0.28949845]]  Left: [[0.9986826]]
Right: [[0.89800835]]  Left: [[0.9820195]]
Right: [[0.08646941]]  Left: [[0.5997515]]
Right: [[0.2583447]]  Left: [[0.9998034]]
Right: [[0.34543616]]  Left: [[0.05703646]]
Right: [[0.8832901]]  Left: [[0.10451621]]
Right: [[0.65868855]]  Left: [[0.3258615

In [31]:
print(stop - start)

18.42406940460205


In [32]:
frame_count/(stop-start)

3.4194400062487587